# Technical Assesment
Author: [Sébastien Lozano Forero](https://www.linkedin.com/in/sebastienlozanoforero/)

April, 2022. 

This project is based on an technical assessment used by a Colombian bank to fill the position of Data Scientist. Data was modified shared in a modified version not containing any identification information. Currency within this project is set to Colombian pesos. 

# Loading libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
import xgboost as xgb
import pickle

from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from pandas_profiling import ProfileReport


from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, metrics, model_selection, svm
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
# from boruta import BorutaPy

# Data Loading

In [ ]:
data = pd.read_csv('Base_campana_DC.csv',encoding = "ISO-8859-1", sep =';',parse_dates=['fecha_ult_desembolso'], 
                  dtype = {
                      'Id_Cliente':str,
                      'Ciudad':str,
                      ' Patrimonio ':int,
                      ' Ingresos_Mensuales ':int,
                      'No_hijos':int,
                      'oficina':str,
                      'monto_credito':int,
                      'tasa':float,
                      'saldo_capital':int,
                      'Antiguedad_en_meses':int,
                      'Max_dias_mora':int,
                      'plazo_dias':int,
                      'Edad':int,
                      'Acepta_Campaña':str,
                  })


## Checking variables names

In [ ]:
data.columns

In [ ]:
data = data.rename(columns = {' Ingresos_Mensuales ':'Ingresos_Mensuales',
                              ' Patrimonio ':'Patrimonio'})

## Chequing Data types

In [ ]:
data.dtypes

# Exploratory and Descriptiva Analysis (EDA)

In [ ]:
# prof = ProfileReport(data)
# prof.to_file(output_file='output.html')

In [ ]:
num_attributes = data.select_dtypes( include=['int32', 'float64'] )
cat_attributes = data.select_dtypes( exclude=['int32', 'float64', 'datetime64[ns]'] )

In [ ]:
# sns.pairplot(num_attributes)

In [ ]:
data['mes_ult_desembolso'] = data['fecha_ult_desembolso'].dt.month
data['semana_ult_desembolso'] = data['fecha_ult_desembolso'].dt.weekofyear

## Data Filtering

In [ ]:
data['No_hijos'].value_counts()

Queremos filtrar valores muy altos en ingresos o números de hijos (hay un cliente con ingresos de 26 millones al mes y cuatro hijos).

In [ ]:
data = data[(data['Ingresos_Mensuales'] != data['Ingresos_Mensuales'].max())&
            (data['No_hijos'] <3)]

## Data preparation

In [ ]:
data.columns

In [ ]:
X = data[[ 'Patrimonio', 'Ingresos_Mensuales', 'No_hijos',
       'monto_credito', 'tasa',
       'saldo_capital', 'saldo_Ahorro', 'Antiguedad_en_meses', 'Max_dias_mora',
       'plazo_dias', 'Edad', 'mes_ult_desembolso']]
y  = data['Acepta_Campaña']

In [ ]:
ciudad = pd.get_dummies(data['Ciudad'], prefix='C')

In [ ]:
X= X.join(ciudad)

In [ ]:
mms = MinMaxScaler()

X['Patrimonio'] = mms.fit_transform( X[['Patrimonio']].values )
X['Ingresos_Mensuales'] = mms.fit_transform( X[['Ingresos_Mensuales']].values )
X['No_hijos'] = mms.fit_transform( X[['No_hijos']].values )
X['monto_credito'] = mms.fit_transform( X[['monto_credito']].values )
X['tasa'] = mms.fit_transform( X[['tasa']].values )
X['saldo_capital'] = mms.fit_transform( X[['saldo_capital']].values )
X['saldo_Ahorro'] = mms.fit_transform( X[['saldo_Ahorro']].values )
X['Antiguedad_en_meses'] = mms.fit_transform( X[['Antiguedad_en_meses']].values )
X['Max_dias_mora'] = mms.fit_transform( X[['Max_dias_mora']].values )
X['Edad'] = mms.fit_transform( X[['Edad']].values )
X['plazo_dias'] = mms.fit_transform( X[['plazo_dias']].values )
X['mes_ult_desembolso'] = mms.fit_transform( X[['mes_ult_desembolso']].values )
# X['semana_ult_desembolso'] = mms.fit_transform( X[['semana_ult_desembolso']].values )

# Machine Learning model 

## Data Splitting

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Baseline Clasiffier

In [ ]:
dummy_clf = DummyClassifier(random_state=42).fit(X_train, y_train)
y_pred_clf = dummy_clf.predict(X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_clf)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión Clasificador Aleatorio\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(dummy_clf, X_test, y_test) 

In [ ]:
scores = cross_val_score(dummy_clf, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## k-nearest neighbors 


In [ ]:
dummy_KNN = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
y_pred_knn = dummy_KNN.predict(X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_knn)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión Regresión Logística\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(dummy_KNN, X_test, y_test) 

In [ ]:
scores = cross_val_score(dummy_KNN, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Logistic Regression

In [ ]:
reg_log = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_reg_log = reg_log.predict(X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_reg_log)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión Regresión Logística\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(reg_log, X_test, y_test) 

In [ ]:
scores = cross_val_score(reg_log, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Random Forest

In [ ]:
rf =RandomForestClassifier(max_depth=2, random_state=0).fit( X_train, y_train )

# prediction
y_pred_rf = rf.predict( X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_rf)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión Bosque Aleatorio\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(rf, X_test, y_test) 

In [ ]:
scores = cross_val_score(rf, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Naive Bayes

In [ ]:
nb =GaussianNB().fit( X_train, y_train )

# prediction
y_pred_NB = nb.predict( X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_NB)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión Naive Bayes\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(nb, X_test, y_test) 

In [ ]:
scores = cross_val_score(nb, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## XGBoost 

In [ ]:
xgb_cl = xgb.XGBClassifier().fit( X_train, y_train )

y_pred_xgb = xgb_cl.predict(X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_xgb)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión XGboost\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(xgb_cl, X_test, y_test) 

In [ ]:
scores = cross_val_score(xgb_cl, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Summary of results

In [ ]:
scores_bln = cross_val_score(dummy_clf, X=X_train, y=y_train, cv=10, n_jobs=1)
scores_knn = cross_val_score(dummy_KNN, X=X_train, y=y_train, cv=10, n_jobs=1)
scores_log = cross_val_score(reg_log, X=X_train, y=y_train, cv=10, n_jobs=1)
scores_rf = cross_val_score(rf, X=X_train, y=y_train, cv=10, n_jobs=1)
scores_nb = cross_val_score(nb, X=X_train, y=y_train, cv=10, n_jobs=1)
scores_xgb = cross_val_score(xgb_cl, X=X_train, y=y_train, cv=10, n_jobs=1)


In [ ]:
res_baseline = pd.DataFrame( {'Model Name': 'Baseline', 
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_bln),np.std(scores_bln))},index=[0] ) 
res_KNN = pd.DataFrame( {'Model Name': 'KNN', 
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_knn),np.std(scores_knn))},index=[0] ) 
res_lr = pd.DataFrame( {'Model Name': 'Logistic Regression',
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_log),np.std(scores_log))},index=[0] ) 
res_rf = pd.DataFrame( {'Model Name': 'Random Forest',
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_rf),np.std(scores_rf))},index=[0] ) 
res_nb = pd.DataFrame( {'Model Name': 'Naive Bayes', 
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_nb),np.std(scores_nb))},index=[0] ) 
res_xgb = pd.DataFrame( {'Model Name': 'XGBoost', 
                'Accuracy':  '%.3f +/- %.3f' % (np.mean(scores_xgb),np.std(scores_xgb))},index=[0] )
Accuracy = pd.concat([res_baseline,res_KNN,res_lr,res_rf,res_nb,res_xgb])

In [ ]:
Accuracy.sort_values('Accuracy', ascending = False)

Thus, the model XGboost is going to be selected for forecasting. But first, let see if a better version if achivable by means of an hyperparameter tuning process

## Model Hyperparameter Tuning

In [ ]:
dtrain = xgb.DMatrix(X_test, label=y_test)

def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
                'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]


In [ ]:
#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                            'gamma': (0, 1),
                                            'learning_rate':(0,1),
                                             'n_estimators':(100,120)})

In [ ]:
xgb_bo.maximize(n_iter=5, init_points=8, acq='ei')

In [ ]:
#Extracting the best parameters
params = xgb_bo.max['params']
print(params)

In [ ]:
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])

#Initialize an XGBClassifier with the tuned parameters and fit the training data

classifier2 = xgb.XGBClassifier(**params).fit(X_test, y_test)

In [ ]:
pickle.dump( classifier2, open( 'model/final_model.pkl', 'wb' ) )

In [ ]:
y_pred_xgb_param = classifier2.predict(X_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_xgb_param)

print(cf_matrix)

In [ ]:
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Matriz de confusión XGboost\n\n');
ax.set_xlabel('\n Valores predichos ')
ax.set_ylabel('Valores observados ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
metrics.plot_roc_curve(classifier2, X_test, y_test) 

In [ ]:
scores = cross_val_score(classifier2, X=X_train, y=y_train, cv=10, n_jobs=1)
 
print('Puntajes obtenidos: %s' % scores)
print('Precisión de validación cruzada: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Using the model to predict new data

In [4]:
nueva = pd.read_csv('Base_nueva_DC.csv',encoding = "ISO-8859-1", sep =';',parse_dates=['fecha_ult_desembolso'], 
                  dtype = {
                      'Id_Cliente':str,
                      'Ciudad':str,
                      ' Patrimonio ':int,
                      ' Ingresos_Mensuales ':int,
                      'No_hijos':int,
                      'oficina':str,
                      'monto_credito':int,
                      'tasa':float,
                      'saldo_capital':int,
                      'Antiguedad_en_meses':int,
                      'Max_dias_mora':int,
                      'plazo_dias':int,
                      'Edad':int
                  })


In [5]:
nueva['mes_ult_desembolso'] = nueva['fecha_ult_desembolso'].dt.month
nueva = nueva.rename(columns = {' Ingresos_Mensuales ':'Ingresos_Mensuales',
                              ' Patrimonio ':'Patrimonio'})

In [6]:
nueva

,Id_Cliente,Ciudad,Patrimonio,Ingresos_Mensuales,No_hijos,oficina,fecha_ult_desembolso,monto_credito,tasa,saldo_capital,saldo_Ahorro,Antiguedad_en_meses,Max_dias_mora,plazo_dias,Edad,mes_ult_desembolso
0,427658,Cartagena,745968,1614094,0,4024,2017-12-30,1500000,33.93,1357387,33438,12,0,900,65,12
1,427659,Barranquilla,834415,920234,1,4011,2017-12-18,1500000,33.93,392262,0,12,0,540,40,12
2,427670,Barranquilla,1164858,1568317,1,7032,2018-02-17,1064170,0.30,211724,0,12,0,540,36,2
3,427671,Cali,1187587,796660,1,7032,2017-12-25,2338552,0.30,828993,0,12,0,720,47,12
4,427720,Cartagena,2840488,1474771,0,7025,2018-01-23,2385560,33.93,1997956,0,12,0,720,56,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,428626,Cartagena,37964203,477534,1,4003,2017-11-29,9000000,33.93,7810084,18603,120,0,1080,30,11
1496,428844,Barranquilla,46168677,1516403,0,4003,2017-11-29,8900000,33.93,7810084,18603,120,0,1080,44,11
1497,428405,Bogotá,28739705,848160,1,4030,2017-12-15,9000000,33.93,9000000,19713,120,0,720,41,12
1498,428623,Cali,37858892,1041188,2,4030,2017-12-15,9000000,33.93,9000000,19713,120,0,720,48,12


In [7]:
X = nueva[[ 'Patrimonio', 'Ingresos_Mensuales', 'No_hijos',
       'monto_credito', 'tasa',
       'saldo_capital', 'saldo_Ahorro', 'Antiguedad_en_meses', 'Max_dias_mora',
       'plazo_dias', 'Edad', 'mes_ult_desembolso']]

In [8]:
ciudad = pd.get_dummies(nueva['Ciudad'], prefix='C')

In [9]:
X= X.join(ciudad)

In [10]:
X.head().T

,0,1,2,3,4
Patrimonio,745968.00,834415.00,1164858.0,1187587.0,2840488.00
Ingresos_Mensuales,1614094.00,920234.00,1568317.0,796660.0,1474771.00
No_hijos,0.00,1.00,1.0,1.0,0.00
monto_credito,1500000.00,1500000.00,1064170.0,2338552.0,2385560.00
tasa,33.93,33.93,0.3,0.3,33.93
saldo_capital,1357387.00,392262.00,211724.0,828993.0,1997956.00
saldo_Ahorro,33438.00,0.00,0.0,0.0,0.00
Antiguedad_en_meses,12.00,12.00,12.0,12.0,12.00
Max_dias_mora,0.00,0.00,0.0,0.0,0.00
plazo_dias,900.00,540.00,540.0,720.0,720.00


In [14]:
import joblib
mms = MinMaxScaler()
mms.fit_transform( X[['Patrimonio']].values )
joblib.dump(mms, 'parameter/scaler.gz')

# pickle.dump( mms, open( 'parameter/Patrimonio.pkl', 'wb') )
mms_test = joblib.load('parameter/scaler.gz')


MinMaxScaler()


In [16]:
mms_test.transform(X[['Patrimonio']].values)

array([[0.00527939],
       [0.00683639],
       [0.01265344],
       ...,
       [0.49807511],
       [0.65860731],
       [0.8035164 ]])

In [ ]:
mms1 = MinMaxScaler()
X['Patrimonio'] = mms1.fit_transform( X[['Patrimonio']].values )
joblib.dump(mms1, 'parameter/Patrimonio.gz')

In [ ]:
mms = MinMaxScaler()

X['Ingresos_Mensuales'] = mms.fit_transform( X[['Ingresos_Mensuales']].values )
joblib.dump(mms, 'parameter/Ingresos_Mensuales.gz')
X['No_hijos'] = mms.fit_transform( X[['No_hijos']].values )
joblib.dump(mms, 'parameter/No_hijos.gz')
X['monto_credito'] = mms.fit_transform( X[['monto_credito']].values )
joblib.dump(mms, 'parameter/monto_credito.gz')
X['tasa'] = mms.fit_transform( X[['tasa']].values )
joblib.dump(mms, 'parameter/tasa.gz')
X['saldo_capital'] = mms.fit_transform( X[['saldo_capital']].values )
joblib.dump(mms, 'parameter/saldo_capital.gz')
X['saldo_Ahorro'] = mms.fit_transform( X[['saldo_Ahorro']].values )
joblib.dump(mms, 'parameter/saldo_Ahorro.gz')
X['Antiguedad_en_meses'] = mms.fit_transform( X[['Antiguedad_en_meses']].values )
joblib.dump(mms, 'parameter/Antiguedad_en_meses.gz')
X['Max_dias_mora'] = mms.fit_transform( X[['Max_dias_mora']].values )
joblib.dump(mms, 'parameter/Max_dias_mora.gz')
X['Edad'] = mms.fit_transform( X[['Edad']].values )
joblib.dump(mms, 'parameter/Edad.gz')
X['plazo_dias'] = mms.fit_transform( X[['plazo_dias']].values )
joblib.dump(mms, 'parameter/plazo_dias.gz')
X['mes_ult_desembolso'] = mms.fit_transform( X[['mes_ult_desembolso']].values )
joblib.dump(mms, 'parameter/mes_ult_desembolso.gz')
# X['semana_ult_desembolso'] = mms.fit_transform( X[['semana_ult_desembolso']].values )

In [ ]:
joblib.load('parameter/Patrimonio.gz').fit_transform(X[['Patrimonio']].values)

In [ ]:
test = open('parameter/Patrimonio.gz', 'rb')
joblib.load(test)#.fit_transform( X[['Patrimonio']].values )

In [ ]:
X.head().T

In [ ]:
y_xgb = classifier2.predict(X)
y_xgb_proba = classifier2.predict_proba(X)
nueva = nueva.join(pd.DataFrame(y_xgb_proba))
# nueva = nueva.join(pd.DataFrame(y_xgb))

In [ ]:
pd.DataFrame(y_xgb).rename(columns = {0:'Aceptará Campaña'}).value_counts('Aceptará Campaña')

In [ ]:
nueva = nueva.rename(columns={0:'Prob_no_aceptar',1:'Prob_aceptar'})

In [ ]:
nueva.sort_values('Prob_aceptar', ascending = False)